In [1]:
import pandas as pd
df = pd.read_csv('D:\\min project\\traffic_stops - traffic_stops_with_vehicle_number.csv')

C:\Users\Administrator\AppData\Local\Temp\ipykernel_12768\2758829557.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('D:\\min project\\traffic_stops - traffic_stops_with_vehicle_number.csv')


In [2]:
print("✅ Data loaded successfully!")
print(f"Initial shape: {df.shape}")
print("Columns:", list(df.columns))

✅ Data loaded successfully!
Initial shape: (65538, 16)
Columns: ['stop_date', 'stop_time', 'country_name', 'driver_gender', 'driver_age_raw', 'driver_age', 'driver_race', 'violation_raw', 'violation', 'search_conducted', 'search_type', 'stop_outcome', 'is_arrested', 'stop_duration', 'drugs_related_stop', 'vehicle_number']


In [3]:
# Remove columns that have all values as NaN (empty)
df = df.dropna(axis=1, how='all')


In [4]:
print("\n✅ Columns with all NaN removed.")
print(f"Shape after removal: {df.shape}")
print("Remaining Columns:", list(df.columns))



✅ Columns with all NaN removed.
Shape after removal: (65538, 16)
Remaining Columns: ['stop_date', 'stop_time', 'country_name', 'driver_gender', 'driver_age_raw', 'driver_age', 'driver_race', 'violation_raw', 'violation', 'search_conducted', 'search_type', 'stop_outcome', 'is_arrested', 'stop_duration', 'drugs_related_stop', 'vehicle_number']


In [5]:

# Replace NaN with None (SQL-compatible)
df = df.where(pd.notnull(df), None)


In [6]:
#  View the cleaned data
print("\n✅ Sample cleaned data:")
print(df.head())


✅ Sample cleaned data:
    stop_date stop_time country_name driver_gender  driver_age_raw  \
0  2020-01-01   0:00:00       Canada             M              59   
1  2020-01-01   0:01:00        India             M              35   
2  2020-01-01   0:02:00          USA             M              26   
3  2020-01-01   0:03:00       Canada             M              26   
4  2020-01-01   0:04:00       Canada             M              62   

   driver_age driver_race     violation_raw violation  search_conducted  \
0          19       Asian     Drunk Driving  Speeding              True   
1          58       Other             Other     Other             False   
2          76       Black  Signal Violation  Speeding             False   
3          76       Black          Speeding       DUI              True   
4          75       Other          Speeding     Other             False   

      search_type stop_outcome  is_arrested stop_duration  drugs_related_stop  \
0  Vehicle Search      

In [7]:
# Step 5: Save cleaned data to new CSV (optional)
df.to_csv('cleaned_traffic_stops.csv', index=False)
print("\n✅ Cleaned data saved as 'cleaned_traffic_stops2.csv'.")


✅ Cleaned data saved as 'cleaned_traffic_stops2.csv'.


In [8]:
# Convert stop_date to DD-MM-YYYY format
df['stop_date'] = pd.to_datetime(df['stop_date'], errors='coerce').dt.strftime('%d-%m-%Y')

In [9]:
#  Convert stop_time to HH:MM:SS format
df['stop_time'] = pd.to_datetime(df['stop_time'], format='%H:%M:%S', errors='coerce').dt.strftime('%H:%M:%S')


In [10]:
df.to_csv('formatted_traffic_stops.csv', index=False)
print("\n✅ Formatted data saved as 'formatted_traffic_stops.csv2'.")


✅ Formatted data saved as 'formatted_traffic_stops.csv2'.


In [11]:
print(df.head())

    stop_date stop_time country_name driver_gender  driver_age_raw  \
0  01-01-2020  00:00:00       Canada             M              59   
1  01-01-2020  00:01:00        India             M              35   
2  01-01-2020  00:02:00          USA             M              26   
3  01-01-2020  00:03:00       Canada             M              26   
4  01-01-2020  00:04:00       Canada             M              62   

   driver_age driver_race     violation_raw violation  search_conducted  \
0          19       Asian     Drunk Driving  Speeding              True   
1          58       Other             Other     Other             False   
2          76       Black  Signal Violation  Speeding             False   
3          76       Black          Speeding       DUI              True   
4          75       Other          Speeding     Other             False   

      search_type stop_outcome  is_arrested stop_duration  drugs_related_stop  \
0  Vehicle Search       Ticket         True    

In [12]:
import psycopg2

# Connect to PostgreSQL
conn = psycopg2.connect(
    dbname="traffic_data",
    user="postgres",
    password="9791804453",
    host="localhost",
    port="5432"
)

cur = conn.cursor()
print("✅ Connected to PostgreSQL.")

✅ Connected to PostgreSQL.


In [13]:
# SQL Query: Top 10 vehicles in drug-related stops
query = '''
SELECT vehicle_number, COUNT(*) AS stop_count
FROM traffic_stops
WHERE drugs_related_stop = TRUE
GROUP BY vehicle_number
ORDER BY stop_count DESC
LIMIT 10;
'''

# Execute and fetch
cur.execute(query)
results = cur.fetchall()


In [14]:
# Display
print("\nTop 10 Vehicles in Drug-Related Stops:")
for row in results:
    print(f"Vehicle: {row[0]}, Stops: {row[1]}")



Top 10 Vehicles in Drug-Related Stops:
Vehicle: TN16EN4769, Stops: 3
Vehicle: GJ99TA8565, Stops: 3
Vehicle: MH85NR6252, Stops: 3
Vehicle: RJ88QF9639, Stops: 3
Vehicle: WB82DO3310, Stops: 3
Vehicle: TN98SO9644, Stops: 3
Vehicle: RJ69QZ5424, Stops: 3
Vehicle: RJ79BO9666, Stops: 3
Vehicle: TN57CN7350, Stops: 3
Vehicle: TN72BR3512, Stops: 3


In [15]:
##-Which Vehicles Were Most Frequently Searched
query = '''
SELECT vehicle_number, COUNT(*) AS search_count
FROM traffic_stops
WHERE search_conducted = TRUE
GROUP BY vehicle_number
ORDER BY search_count DESC
LIMIT 10;
'''
cur.execute(query)
print("\nTop 10 Vehicles Most Frequently Searched:")
for row in cur.fetchall():
    print(row)



Top 10 Vehicles Most Frequently Searched:
('TN24KB4148', 3)
('TN16EN4769', 3)
('MH85NR6252', 3)
('KA36NX6664', 3)
('WB82DO3310', 3)
('RJ14TR4444', 3)
('RJ49OQ1668', 3)
('MH21PZ2993', 3)
('RJ69QZ5424', 3)
('TN72BR3512', 3)


In [16]:
##-Which Driver Age Group Had the Highest Arrest Rate
query = '''
SELECT driver_age, COUNT(*) AS arrest_count
FROM traffic_stops
WHERE is_arrested = TRUE
GROUP BY driver_age
ORDER BY arrest_count DESC
LIMIT 10;
'''
cur.execute(query)
print("\nTop 10 Driver Age Groups with Highest Arrest Rate:")
for row in cur.fetchall():
    print(row)



Top 10 Driver Age Groups with Highest Arrest Rate:
(28, 1740)
(26, 1665)
(34, 1662)
(60, 1659)
(38, 1650)
(63, 1650)
(62, 1650)
(21, 1647)
(56, 1644)
(53, 1644)


In [17]:
##-Gender Distribution of Drivers Stopped in Each Country
query = '''
SELECT country_name, driver_gender, COUNT(*) AS count
FROM traffic_stops
GROUP BY country_name, driver_gender
ORDER BY country_name, count DESC;
'''
cur.execute(query)
print("\nGender Distribution of Drivers Stopped in Each Country:")
for row in cur.fetchall():
    print(row)



Gender Distribution of Drivers Stopped in Each Country:
('Canada', 'F', 32991)
('Canada', 'M', 32733)
('India', 'F', 33129)
('India', 'M', 32865)
('USA', 'F', 32523)
('USA', 'M', 32373)


In [18]:
##-Which Race and Gender Combination Has the Highest Search Rate
query = '''
SELECT driver_race, driver_gender, COUNT(*) AS search_count
FROM traffic_stops
WHERE search_conducted = TRUE
GROUP BY driver_race, driver_gender
ORDER BY search_count DESC
LIMIT 10;
'''
cur.execute(query)
print("\nTop 10 Race and Gender Combinations with Highest Search Rate:")
for row in cur.fetchall():
    print(row)



Top 10 Race and Gender Combinations with Highest Search Rate:
('Asian', 'F', 10230)
('Hispanic', 'F', 9978)
('Other', 'F', 9882)
('Black', 'M', 9795)
('White', 'M', 9786)
('White', 'F', 9771)
('Asian', 'M', 9717)
('Other', 'M', 9714)
('Hispanic', 'M', 9636)
('Black', 'F', 9594)


In [19]:
##-What Time of Day Sees the Most Traffic Stops
query = '''
SELECT EXTRACT(HOUR FROM stop_time::time) AS hour_of_day, COUNT(*) AS stop_count
FROM traffic_stops
GROUP BY hour_of_day
ORDER BY stop_count DESC
LIMIT 10;
'''
cur.execute(query)
print("\nTop 10 Hours with Most Traffic Stops:")
for row in cur.fetchall():
    print(row)



Top 10 Hours with Most Traffic Stops:
(None, 131076)
(Decimal('0'), 2760)
(Decimal('4'), 2760)
(Decimal('2'), 2760)
(Decimal('9'), 2760)
(Decimal('3'), 2760)
(Decimal('6'), 2760)
(Decimal('11'), 2760)
(Decimal('7'), 2760)
(Decimal('10'), 2760)


In [20]:
##-Average Stop Duration for Different Violations
query = '''
SELECT violation, AVG(
    CASE 
        WHEN stop_duration = '<5 min' THEN 5
        WHEN stop_duration = '6-15 min' THEN 10
        WHEN stop_duration = '16-30 min' THEN 23
        ELSE 45
    END
) AS avg_duration
FROM traffic_stops
GROUP BY violation
ORDER BY avg_duration DESC;
'''
cur.execute(query)
print("\nAverage Stop Duration for Different Violations:")
for row in cur.fetchall():
    print(row)



Average Stop Duration for Different Violations:
('DUI', Decimal('45.0000000000000000'))
('Other', Decimal('45.0000000000000000'))
('Seatbelt', Decimal('45.0000000000000000'))
('Signal', Decimal('45.0000000000000000'))
('Speeding', Decimal('45.0000000000000000'))


In [21]:
## Are Night Stops More Likely to Lead to Arrests
query = '''
SELECT 
    CASE 
        WHEN EXTRACT(HOUR FROM stop_time::time) BETWEEN 0 AND 5 THEN 'Night'
        ELSE 'Day'
    END AS time_of_day,
    COUNT(CASE WHEN is_arrested = TRUE THEN 1 END) * 100.0 / COUNT(*) AS arrest_rate
FROM traffic_stops
GROUP BY time_of_day;
'''
cur.execute(query)
print("\nArrest Rate During Night vs Day:")
for row in cur.fetchall():
    print(row)




Arrest Rate During Night vs Day:
('Day', Decimal('50.1999400179946016'))
('Night', Decimal('49.2210144927536232'))


In [22]:
##-Violations Most Associated with Searches or Arrests
query = '''
SELECT violation, 
       COUNT(CASE WHEN search_conducted = TRUE THEN 1 END) AS search_count,
       COUNT(CASE WHEN is_arrested = TRUE THEN 1 END) AS arrest_count
FROM traffic_stops
GROUP BY violation
ORDER BY search_count DESC, arrest_count DESC
LIMIT 10;
'''
cur.execute(query)
print("\nViolations Most Associated with Searches or Arrests:")
for row in cur.fetchall():
    print(row)



Violations Most Associated with Searches or Arrests:
('Other', 19728, 19548)
('Speeding', 19719, 19794)
('Seatbelt', 19686, 19719)
('Signal', 19527, 19647)
('DUI', 19443, 19830)


In [23]:
##-Violations Most Common Among Younger Drivers (<25)
query = '''
SELECT violation, COUNT(*) AS count
FROM traffic_stops
WHERE driver_age < 25
GROUP BY violation
ORDER BY count DESC
LIMIT 10;
'''
cur.execute(query)
print("\nViolations Most Common Among Younger Drivers:")
for row in cur.fetchall():
    print(row)



Violations Most Common Among Younger Drivers:
('Speeding', 4428)
('Signal', 4281)
('Other', 4266)
('Seatbelt', 4260)
('DUI', 4077)


In [ ]:
##-Violations Rarely Leading to Search or Arrest

def run_query(query):
    try:
        conn = psycopg2.connect(
            dbname="traffic_data",
            user="postgres",
            password="9791804453",
            host="localhost",
            port="5432"
        )
        cur = conn.cursor()
        cur.execute(query)
        results = cur.fetchall()
        cur.close()
        conn.close()
        return results
    except Exception as e:
        st.error(f"❌ SQL Error: {e}")
        return []


In [35]:
##-Violations Rarely Leading to Search or Arrest
query = '''
SELECT 
    violation,
    COUNT(*) AS total_stops,
    COUNT(CASE WHEN search_conducted = TRUE THEN 1 END) AS search_count,
    COUNT(CASE WHEN is_arrested = TRUE THEN 1 END) AS arrest_count
FROM traffic_stops
GROUP BY violation
ORDER BY total_stops DESC;
'''

run_query(query)



[('Other', 39582, 19728, 19548),
 ('Speeding', 39450, 19719, 19794),
 ('Signal', 39336, 19527, 19647),
 ('DUI', 39225, 19443, 19830),
 ('Seatbelt', 39021, 19686, 19719)]

In [ ]:
##Countries Reporting Highest Rate of Drug-Related Stops
query = '''
SELECT country_name, COUNT(*) AS drug_stop_count
FROM traffic_stops
WHERE drugs_related_stop = TRUE
GROUP BY country_name
ORDER BY drug_stop_count DESC
LIMIT 10;
'''
cur.execute(query)
print("\nCountries with Highest Drug-Related Stop Rates:")
for row in cur.fetchall():
    print(row)



Countries with Highest Drug-Related Stop Rates:
('Canada', 32928)
('India', 32691)
('USA', 32688)


In [ ]:
##-Arrest Rate by Country and Violation
query = '''
SELECT country_name, violation,
       COUNT(*) AS total_stops,
       COUNT(CASE WHEN is_arrested = TRUE THEN 1 END) * 100.0 / COUNT(*) AS arrest_rate
FROM traffic_stops
GROUP BY country_name, violation
ORDER BY arrest_rate DESC
LIMIT 10;
'''
cur.execute(query)
print("\nArrest Rate by Country and Violation:")
for row in cur.fetchall():
    print(row)



Arrest Rate by Country and Violation:
('Canada', 'DUI', 13242, Decimal('51.1327594019030358'))
('India', 'DUI', 13260, Decimal('50.8371040723981900'))
('India', 'Speeding', 13074, Decimal('50.7572280862781092'))
('India', 'Signal', 13332, Decimal('50.6525652565256526'))
('USA', 'Seatbelt', 12723, Decimal('50.5776939401084650'))
('India', 'Seatbelt', 13035, Decimal('50.5638665132336018'))
('Canada', 'Seatbelt', 13263, Decimal('50.4636959963809093'))
('Canada', 'Other', 13113, Decimal('50.2859757492564631'))
('USA', 'Signal', 13023, Decimal('49.9884819166090762'))
('Canada', 'Speeding', 13125, Decimal('49.9428571428571429'))


In [ ]:
##- Countries with Most Stops Where Search Was Conducted
query = '''
SELECT country_name, COUNT(*) AS search_count
FROM traffic_stops
WHERE search_conducted = TRUE
GROUP BY country_name
ORDER BY search_count DESC
LIMIT 10;
'''
cur.execute(query)
print("\nCountries with Most Stops Where Search Was Conducted:")
for row in cur.fetchall():
    print(row)



Countries with Most Stops Where Search Was Conducted:
('Canada', 33060)
('India', 32877)
('USA', 32166)


In [ ]:
##-Driver Violation Trends Based on Age and Race
query = '''
SELECT driver_age, driver_race, violation, COUNT(*) AS count
FROM traffic_stops
GROUP BY driver_age, driver_race, violation
ORDER BY count DESC
LIMIT 10;
'''
cur.execute(query)
print("\nDriver Violation Trends by Age and Race:")
for row in cur.fetchall():
    print(row)



Driver Violation Trends by Age and Race:
(56, 'White', 'DUI', 189)
(61, 'White', 'DUI', 177)
(78, 'Asian', 'DUI', 177)
(71, 'Other', 'Speeding', 177)
(39, 'White', 'Other', 174)
(54, 'Black', 'Other', 174)
(33, 'Other', 'Speeding', 174)
(79, 'Asian', 'Other', 174)
(70, 'Black', 'DUI', 174)
(44, 'Other', 'Signal', 171)


In [ ]:
##-Number of Stops by Year, Month, Hour
query = '''
SELECT 
    EXTRACT(YEAR FROM TO_DATE(stop_date, 'DD-MM-YYYY')) AS year,
    EXTRACT(MONTH FROM TO_DATE(stop_date, 'DD-MM-YYYY')) AS month,
    EXTRACT(HOUR FROM stop_time::time) AS hour,
    COUNT(*) AS stop_count
FROM traffic_stops
GROUP BY year, month, hour
ORDER BY year, month, hour;
'''
cur.execute(query)
print("\nNumber of Stops by Year, Month, Hour:")
for row in cur.fetchall():
    print(row)


InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block


In [ ]:
try:
    cur.execute(query)
    results = cur.fetchall()
    for row in results:
        print(row)
except Exception as e:
    print("❌ Error:", e)
    conn.rollback()  # Rollback the transaction so you can continue


❌ Error: current transaction is aborted, commands ignored until end of transaction block



In [ ]:
query = '''
SELECT 
    EXTRACT(YEAR FROM stop_date) AS year,
    EXTRACT(MONTH FROM stop_date) AS month,
    EXTRACT(HOUR FROM stop_time::time) AS hour,
    COUNT(*) AS stop_count
FROM traffic_stops
WHERE stop_date IS NOT NULL
GROUP BY year, month, hour
ORDER BY year, month, hour;
'''


In [ ]:
def run_query(query):
    try:
        cur.execute(query)
        results = cur.fetchall()
        for row in results:
            print(row)
    except Exception as e:
        print("❌ Error:", e)
        conn.rollback()


In [ ]:
run_query(query)


(Decimal('2020'), Decimal('1'), Decimal('0'), 120)
(Decimal('2020'), Decimal('1'), Decimal('1'), 120)
(Decimal('2020'), Decimal('1'), Decimal('2'), 120)
(Decimal('2020'), Decimal('1'), Decimal('3'), 120)
(Decimal('2020'), Decimal('1'), Decimal('4'), 120)
(Decimal('2020'), Decimal('1'), Decimal('5'), 120)
(Decimal('2020'), Decimal('1'), Decimal('6'), 120)
(Decimal('2020'), Decimal('1'), Decimal('7'), 120)
(Decimal('2020'), Decimal('1'), Decimal('8'), 120)
(Decimal('2020'), Decimal('1'), Decimal('9'), 120)
(Decimal('2020'), Decimal('1'), Decimal('10'), 120)
(Decimal('2020'), Decimal('1'), Decimal('11'), 120)
(Decimal('2020'), Decimal('1'), Decimal('12'), 120)
(Decimal('2020'), Decimal('1'), Decimal('13'), 120)
(Decimal('2020'), Decimal('1'), Decimal('14'), 120)
(Decimal('2020'), Decimal('1'), Decimal('15'), 120)
(Decimal('2020'), Decimal('1'), Decimal('16'), 120)
(Decimal('2020'), Decimal('1'), Decimal('17'), 120)
(Decimal('2020'), Decimal('1'), Decimal('18'), 120)
(Decimal('2020'), Deci

In [ ]:
def run_query(query):
    try:
        cur.execute(query)
        results = cur.fetchall()
        print("\nFormatted Results:")
        for row in results:
            # Convert Decimals to int or str
            formatted_row = tuple(int(value) if isinstance(value, Decimal) else value for value in row)
            print(formatted_row)
    except Exception as e:
        print("❌ Error:", e)
        conn.rollback()


In [ ]:
run_query(query)


Formatted Results:
❌ Error: name 'Decimal' is not defined


In [ ]:
from decimal import Decimal  # <-- Add this line

def run_query(query):
    try:
        cur.execute(query)
        results = cur.fetchall()
        print("\nFormatted Results:")
        for row in results:
            # Convert Decimals to int or str
            formatted_row = tuple(int(value) if isinstance(value, Decimal) else value for value in row)
            print(formatted_row)
    except Exception as e:
        print("❌ Error:", e)
        conn.rollback()


In [ ]:
run_query(query)


Formatted Results:
(2020, 1, 0, 120)
(2020, 1, 1, 120)
(2020, 1, 2, 120)
(2020, 1, 3, 120)
(2020, 1, 4, 120)
(2020, 1, 5, 120)
(2020, 1, 6, 120)
(2020, 1, 7, 120)
(2020, 1, 8, 120)
(2020, 1, 9, 120)
(2020, 1, 10, 120)
(2020, 1, 11, 120)
(2020, 1, 12, 120)
(2020, 1, 13, 120)
(2020, 1, 14, 120)
(2020, 1, 15, 120)
(2020, 1, 16, 120)
(2020, 1, 17, 120)
(2020, 1, 18, 120)
(2020, 1, 19, 120)
(2020, 1, 20, 120)
(2020, 1, 21, 120)
(2020, 1, 22, 120)
(2020, 1, 23, 120)
(2020, 2, 0, 120)
(2020, 2, 1, 120)
(2020, 2, 2, 120)
(2020, 2, 3, 120)
(2020, 2, 4, 120)
(2020, 2, 5, 120)
(2020, 2, 6, 120)
(2020, 2, 7, 120)
(2020, 2, 8, 120)
(2020, 2, 9, 120)
(2020, 2, 10, 120)
(2020, 2, 11, 120)
(2020, 2, 12, 120)
(2020, 2, 13, 120)
(2020, 2, 14, 120)
(2020, 2, 15, 120)
(2020, 2, 16, 120)
(2020, 2, 17, 120)
(2020, 2, 18, 120)
(2020, 2, 19, 120)
(2020, 2, 20, 120)
(2020, 2, 21, 120)
(2020, 2, 22, 120)
(2020, 2, 23, 120)
(2020, 3, 0, 120)
(2020, 3, 1, 120)
(2020, 3, 2, 120)
(2020, 3, 3, 120)
(2020, 3, 4, 120

In [ ]:
##-Violations with High Search and Arrest Rates 
query = '''
SELECT 
    violation,
    COUNT(*) AS total_stops,
    SUM(CASE WHEN search_conducted = TRUE THEN 1 ELSE 0 END) AS search_count,
    ROUND(SUM(CASE WHEN search_conducted = TRUE THEN 1 ELSE 0 END)::decimal * 100 / COUNT(*), 2) AS search_rate,
    SUM(CASE WHEN is_arrested = TRUE THEN 1 ELSE 0 END) AS arrest_count,
    ROUND(SUM(CASE WHEN is_arrested = TRUE THEN 1 ELSE 0 END)::decimal * 100 / COUNT(*), 2) AS arrest_rate,
    RANK() OVER (ORDER BY SUM(CASE WHEN search_conducted = TRUE THEN 1 ELSE 0 END)::decimal / COUNT(*) DESC) AS search_rank,
    RANK() OVER (ORDER BY SUM(CASE WHEN is_arrested = TRUE THEN 1 ELSE 0 END)::decimal / COUNT(*) DESC) AS arrest_rank
FROM traffic_stops
GROUP BY violation
ORDER BY search_rate DESC, arrest_rate DESC;
'''

run_query(query)



Formatted Results:
('Seatbelt', 39021, 19686, 50, 19719, 50, 1, 2)
('Speeding', 39450, 19719, 49, 19794, 50, 2, 3)
('Other', 39582, 19728, 49, 19548, 49, 3, 5)
('Signal', 39336, 19527, 49, 19647, 49, 4, 4)
('DUI', 39225, 19443, 49, 19830, 50, 5, 1)


In [ ]:
##-Driver Demographics by Country (Age, Gender, and Race)
query = '''
SELECT 
    country_name,
    driver_age,
    driver_gender,
    driver_race,
    COUNT(*) AS stop_count
FROM traffic_stops
GROUP BY country_name, driver_age, driver_gender, driver_race
ORDER BY country_name, stop_count DESC;
'''

run_query(query)



Formatted Results:
('Canada', 32, 'F', 'White', 162)
('Canada', 44, 'M', 'White', 147)
('Canada', 52, 'F', 'Asian', 147)
('Canada', 56, 'M', 'White', 144)
('Canada', 31, 'F', 'Black', 141)
('Canada', 54, 'F', 'White', 141)
('Canada', 69, 'F', 'Asian', 141)
('Canada', 70, 'M', 'Black', 138)
('Canada', 29, 'M', 'Hispanic', 138)
('Canada', 60, 'F', 'Other', 138)
('Canada', 44, 'F', 'White', 138)
('Canada', 21, 'F', 'Asian', 138)
('Canada', 26, 'F', 'Black', 138)
('Canada', 80, 'F', 'Other', 138)
('Canada', 64, 'F', 'Black', 138)
('Canada', 56, 'F', 'Hispanic', 138)
('Canada', 76, 'F', 'Other', 135)
('Canada', 50, 'M', 'Other', 135)
('Canada', 23, 'F', 'White', 135)
('Canada', 58, 'F', 'Other', 135)
('Canada', 75, 'M', 'Hispanic', 135)
('Canada', 32, 'M', 'White', 135)
('Canada', 28, 'M', 'White', 135)
('Canada', 55, 'F', 'White', 135)
('Canada', 66, 'F', 'Asian', 135)
('Canada', 49, 'F', 'Other', 135)
('Canada', 74, 'F', 'Other', 132)
('Canada', 54, 'F', 'Hispanic', 132)
('Canada', 25, '

In [ ]:
##Top 5 Violations with Highest Arrest Rates
query = '''
SELECT 
    violation,
    COUNT(*) AS total_stops,
    SUM(CASE WHEN is_arrested = TRUE THEN 1 ELSE 0 END) AS total_arrests,
    ROUND(SUM(CASE WHEN is_arrested = TRUE THEN 1 ELSE 0 END)::decimal * 100 / COUNT(*), 2) AS arrest_rate
FROM traffic_stops
WHERE is_arrested IS NOT NULL
GROUP BY violation
ORDER BY arrest_rate DESC
LIMIT 5;
'''

run_query(query)



Formatted Results:
('DUI', 39225, 19830, 50)
('Seatbelt', 39021, 19719, 50)
('Speeding', 39450, 19794, 50)
('Signal', 39336, 19647, 49)
('Other', 39582, 19548, 49)
